In [3]:
import os
import random
from datetime import date, timedelta
import xarray as xr
from netCDF4 import Dataset, num2date, date2num
import torch
import torch.nn as nn

import time
import numpy as np
import xarray as xr
import cv2

In [10]:
A = np.load("/scratch/iu60/yl3101/DESRGAN/new_crps/save/bias/vfixed0.001crps/2009/model_G_i000001/lead_time0_whole.npy")

In [11]:
b = np.load("/scratch/iu60/yl3101/DESRGAN/new_crps/save/bias/vfixed0.001crps/2009/model_G_i000001/lead_time1_whole.npy")

In [12]:
A / b

/scratch/zv2/yl3101/tmp/ipykernel_1749465/408177834.py:1: RuntimeWarning: invalid value encountered in true_divide
  A / b


array([[0.98614806, 0.99210554, 0.99085814, ...,        nan, 0.99999994,
        1.        ],
       [0.99562186, 0.98203826, 0.9721408 , ..., 1.        , 1.        ,
        1.0000005 ],
       [0.694243  , 0.9819968 , 0.96524715, ..., 0.9999998 , 1.        ,
        0.9999998 ],
       ...,
       [1.5951294 , 2.086515  , 1.5324438 , ..., 0.99999994, 0.9999998 ,
        1.0000001 ],
       [1.5753448 , 2.353936  , 1.5321456 , ..., 1.0000004 , 1.0000002 ,
        0.99999994],
       [1.57879   , 1.9406282 , 1.5175925 , ..., 1.        ,        nan,
               nan]], dtype=float32)

In [8]:
filepath = "/scratch/iu60/yl3101/AGCD_mask_data/"
data1= xr.open_dataset(filepath + "1998-01-12.nc")
data2= xr.open_dataset(filepath + "1998-01-01.nc")
A = []
A.append(data1.isel(time=0)['precip'].values)
A.append(data2.isel(time=0)['precip'].values)

In [20]:
data1

<xarray.Dataset>
Dimensions:  (lat: 413, lon: 267, time: 1)
Coordinates:
  * lat      (lat) float32 -39.2 -39.15 -39.1 -39.05 ... -18.7 -18.65 -18.6
  * lon      (lon) float32 140.6 140.7 140.7 140.8 ... 153.8 153.8 153.8 153.9
  * time     (time) datetime64[ns] 1998-01-12T09:00:00
Data variables:
    precip   (time, lat, lon) float32 ...

In [13]:
A = np.array(A)
B = np.percentile(A, 90, axis = 0)

In [14]:
A

array([[[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [0.       , 0.       , 0.       , ...,       nan,       nan,
               nan],
        [0.       , 0.       , 0.       , ...,       nan,       nan,
               nan],
        [0.       , 0.       , 0.       , ...,       nan,       nan,
               nan]],

       [[      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        [      nan,       nan,       nan, ...,       nan,       nan,
               nan],
        ...,
        [5.5566406, 4.883789 , 4.296875 , ...,       nan,       nan,
               nan],
        [7.708008 , 6.9541016, 6.2646484, ...,       nan,       nan,
   

In [61]:
C1 = [[0.5, 0.2, 0.3],[1, 0.7, 0.1],[0.2, 0.3, 0.5],[0.4, 0.4, 0.7]]
C1 = np.array(C1)
prob_matrix_C1 = (C1 > [0.4, 0.4, 0.3])

In [66]:
(C1 > [0.4, 0.4, 0.3]) * 1

array([[1, 0, 0],
       [1, 1, 0],
       [0, 0, 1],
       [0, 0, 1]])

In [50]:
CC = [[[0.5, 0.2, 0.3],[1, 0.7, 0.1],[0.2, 0.3, 0.5],[0.4, 0.4, 0.7]], [[0.5, 0.7, 0.3],[1, 0.7, 0.1],[0.5, 0.3, 0.1],[0.4, 0.8, 0.2]]]
CC = np.array(CC)
prob_matrix_CC = (CC > [0.4, 0.4, 0.3])

In [51]:
CC.shape

(2, 4, 3)

In [62]:
prob_matrix_C1


array([[ True, False, False],
       [ True,  True, False],
       [False, False,  True],
       [False, False,  True]])

In [69]:
prob_awap = prob_matrix_C1 * 1

In [70]:
prob_awap

array([[1, 0, 0],
       [1, 1, 0],
       [0, 0, 1],
       [0, 0, 1]])

In [52]:
prob_matrix_CC 

array([[[ True, False, False],
        [ True,  True, False],
        [False, False,  True],
        [False, False,  True]],

       [[ True,  True, False],
        [ True,  True, False],
        [ True, False, False],
        [False,  True, False]]])

In [53]:
prob_matrix_CC * 1

array([[[1, 0, 0],
        [1, 1, 0],
        [0, 0, 1],
        [0, 0, 1]],

       [[1, 1, 0],
        [1, 1, 0],
        [1, 0, 0],
        [0, 1, 0]]])

In [67]:
prob_for = np.mean(prob_matrix_CC * 1, axis = 0)

In [68]:
prob_for

array([[1. , 0.5, 0. ],
       [1. , 1. , 0. ],
       [0.5, 0. , 0.5],
       [0. , 0.5, 0.5]])

In [72]:
(prob_for - prob_awap) ** 2

array([[0.  , 0.25, 0.  ],
       [0.  , 0.  , 0.  ],
       [0.25, 0.  , 0.25],
       [0.  , 0.25, 0.25]])

In [73]:
prob_for - prob_awap

array([[ 0. ,  0.5,  0. ],
       [ 0. ,  0. ,  0. ],
       [ 0.5,  0. , -0.5],
       [ 0. ,  0.5, -0.5]])

In [2]:
model_name = 'vfixed0.001crps'
model_num = 'model_G_i000005'
year = '2009'
time = 0
model_brier95 = np.load(
                "/scratch/iu60/yl3101/DESRGAN/new_crps/save/Brier95/" + model_name + "/" + year + '/' + model_num + "/lead_time" + str(time) + "_whole.npy")

In [6]:
model_brier95.shape

(413, 267)

In [11]:
climat = np.load("/scratch/iu60/yl3101/DESRGAN/new_crps/save/climatology/prob99_climatology_2009_all_lead_time_windows_1.npy")

In [13]:
climat.shape

(575, 413, 267)

In [14]:
climat_mean = np.load("/scratch/iu60/yl3101/DESRGAN/new_crps/save/crps_ss/mean_climatology/2009/window1/climat_lead_time_0.npy")

In [15]:
climat_mean

array(nan)

In [16]:
413 * 267

110271

In [4]:
temp = Dataset("/scratch/iu60/yl3101/DESRGAN/vfixed0.001crps/2018/model_G_i000003/e09/2018-12-31_e09.nc")

In [5]:
temp

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(42), lat(413), lon(267)
    variables(dimensions): int64 time(time), float32 lat(lat), float32 lon(lon), float32 pr(time, lat, lon)
    groups: 

In [11]:
original_temp = Dataset("/scratch/iu60/yl3101/Processed_data/e09/da_pr_20091231_e09.nc")

In [12]:
original_temp

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(42), lat(57), lon(24)
    variables(dimensions): int64 time(time), float32 lat(lat), float32 lon(lon), float32 pr(time, lat, lon)
    groups: 

In [13]:
original_temp['pr'][0]

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, 3.279569500591606e-05, ..., --, --, --],
        ...,
        [4.2433402995811775e-05, 2.820269037329126e-05,
         1.4606453078158665e-05, ..., --, --, --],
        [2.3814363885321654e-05, 1.4957363418943714e-05,
         1.7172571460832842e-05, ..., --, --, --],
        [2.0247924112481996e-05, 1.2739752492052503e-05,
         2.616869824123569e-05, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],
  fill_value=nan,
  dtype=float32)

In [14]:
original_temp['pr'][1]

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, 1.8111235249307356e-06, ..., --, --, --],
        ...,
        [0.00015104233170859516, 0.00021064220345579088,
         0.00025501110940240324, ..., --, --, --],
        [0.00020082919218111783, 0.00024715959443710744,
         0.0002879980602301657, ..., --, --, --],
        [0.00024956720881164074, 0.00025643533444963396,
         0.0002767397672869265, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True, False, ...,  True,  True,  True],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]],
  fill_value=nan,
  dtype=float32)

In [17]:
original_access_data = Dataset("/g/data/ux62/access-s2/hindcast/raw_model/atmos/pr/daily/e09/da_pr_19811231_e09.nc")
original_access_data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    model_id: ACCESS-S2
    distribution_statement: The data are subject to the distribution terms of use outlined in the Bureau of Meteorology's Copyright Notice http://www.bom.gov.au/other/copyright.shtml
    creator_name: Bureau of Meteorology
    copyright: (C) Copyright Commonwealth of Australia (2017), Bureau of Meteorology (ABN 92637 533532), see http://www.bom.gov.au/other/copyright.shtml for terms and conditions of reuse
    acknowledgement: Australian Bureau of Meteorology, Commonwealth of Australia
    title: ACCESS-S2
    id: access-s2
    author: Bureau of Meteorology
    Conventions: CF-1.6
    source: ACCESS-S2
    publisher_url: http://bom.gov.au
    keywords: multi-week, seasonal
    creator_url: http://bom.gov.au
    institute_id: ABOM
    publisher_name: Bureau of Meteorology
    institution: Australian Bureau of Meteorology
    naming_authority: au.gov.bom
    disclaimer: The data 